# Vorverarbeitung

Das notebook widmet sich dem Einlesen, Vorverarbeiten und Zusammenführen der verfügbaren Daten, womit insbesondere die digitalisierten Texte und die Annotationsdaten gemeint sind. Ergebnis ist das DataFrame *meta*, in dem alle verfügbaren und für die Analysen relevanten Daten enthalten sind.

# Import

In [1]:
import glob
import lxml.etree as ET
import copy
import pandas as pd
import numpy as np
from scipy import stats
import re
from tqdm.notebook import *

In [2]:
import spacy
spacy_de_core_news_md = spacy.load('de_core_news_md')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


# Korpustexte einlesen

In [4]:
class text:
    def __init__(self, id = "", source = "", author = "", title = "", content = ""):
        self.id = id
        self.source = source
        self.author = author
        self.title = title
        self.content = content   

In [5]:
def corpus_load_tei(path, load_only_texts_with_ids = False):
    ns = {"TEI" : "http://www.tei-c.org/ns/1.0"}
    corpus = []
    
    files_list = sorted(glob.glob(path))
    
    for i in files_list:
        anthology = ET.parse(open(i, "r"))
        nodes = anthology.findall(".//TEI:TEI", ns)
                                
        for j in nodes:
            # id
            this_id_raw = j.findall(".//TEI:teiHeader/TEI:fileDesc/TEI:titleStmt/TEI:title/TEI:idno", ns)            
            this_id = this_id_raw[-1].text if len(this_id_raw) > 0 and this_id_raw[-1].text is not None else "NoID" # falls keine ID vorhanden ist
            
            # source: Name der Anthologie
            this_source = i.rsplit('/', 1)[-1][0:-4]
            
            # author
            this_author_raw = j.findall(".//TEI:teiHeader/TEI:fileDesc/TEI:titleStmt/TEI:author", ns) 
            this_author = this_author_raw[-1].text if len(this_author_raw) > 0 and this_author_raw[-1].text is not None else "NoAuthor" # falls keine Autor:in vorhanden ist
    
            # content: Alle lines in Listenform
            this_content = [k.text for k in j.findall(".//TEI:text/TEI:body/TEI:div//TEI:l", ns)]
            # this_content = this_content + [k.text for k in j.findall(".//TEI:text/TEI:body/TEI:div//TEI:p", ns)]
            if len(this_content) == 0: this_content = float('NaN')
            
            # title
            this_title_raw = j.findall(".//TEI:teiHeader/TEI:fileDesc/TEI:titleStmt/TEI:title", ns)
            if len(this_title_raw) == 0 or this_title_raw[-1].text is None: # kein Titel vorhanden
                this_title = "NoTitle [" + this_content[0] + "]"
            elif this_title_raw[-1].text == " ": # Titel nur ' '
                this_title = "NoTitle [" + this_content[0] + "]"
            else: # Titel normal
                this_title = this_title_raw[-1].text

            this_text = text(this_id, this_source, this_author, this_title, this_content)
            
            if load_only_texts_with_ids and this_id != "NoID":
                corpus.append(this_text)
            if load_only_texts_with_ids == False:
                corpus.append(this_text)
    
    return corpus

In [6]:
def corpus_load_txt(path):
    corpus = []
    
    files_list = sorted(glob.glob(path))

    for this_path in files_list:    

        # this_id = re.findall("[a-zA-Z]*\\.*\d+\\.[a-zA-Z]+\\.[^ ]+", this_path)
        this_id = re.findall("[0-9]{4}[^ ]*", this_path) # four digits, then everything until first space
        if len(this_id) > 0: 
            this_id = this_id[0]
            this_id = this_id.replace('_', '/') # for re-import
        else:
            this_id = 'NoID'
        
        this_source = re.sub("\\.[0-9][0-9][0-9][^A-Za-z]*$", "", this_id)
        
        this_author = re.findall("(?<= – )(.*)(?=.–)", this_path)[0]
        
        this_title = re.findall("(?<=\\D – )(.*)(?=\\.txt)", this_path)[0]
        
        this_content = open(this_path, 'r').read().splitlines()

        this_text = text(this_id, this_source, this_author, this_title, this_content)

        corpus.append(this_text)
    
    return corpus

In [7]:
def normalize (texts, 
               filter_none = False, 
               lemmatize = False,
               lower = False, 
               remove_punctuation = False):
    
    if lemmatize == True: 
        print("normalize (incl. lemmatizing) ...")
        loop_range = trange(len(texts))
    else: 
        loop_range = range(len(texts))
        
    for i in loop_range:
        if texts[i].content != [] and str(texts[i].content) != 'nan':
            if filter_none:
                texts[i].content = list(filter(None, texts[i].content)) # delete blank lines
            if lemmatize:
                texts[i].content = [lemmatizePhrase(spacy_de_core_news_md, x) for x in texts[i].content] # lemmatize
            if lower:
                texts[i].content = [x.lower() for x in texts[i].content] # lower
            if remove_punctuation:
                texts[i].content = [re.sub(r'[^\w\s]','',x) for x in texts[i].content] # remove punctuation

    return texts

In [8]:
def lemmatizePhrase(lemmatizer, phrase):
    spacy_phrase = lemmatizer(phrase)
    lemmatized_phrase = ' '.join([x.lemma_ for x in spacy_phrase])
    return lemmatized_phrase

In [9]:
def export_corpus_to_txt (corpus, path_to_folder):
    for text in corpus:
        filename = text.id + " – " + text.author + " – " + text.title
        filename = filename.replace('/', '_')
        filename = filename[:200]
        with open(f"{path_to_folder}{filename}.txt", 'w') as f:
            if type(text.content) == float: text.content = ['']
            for line in text.content:
                f.write(f"{line}\n")

In [10]:
# corpus_anth = corpus_load_tei("../resources/corpus_anth_tei/*", load_only_texts_with_ids = True)
# corpus_add = corpus_load_txt("../resources/corpus_add_txt/*")

In [11]:
# Zeilen ohne Inhalt herausfiltern
# corpus_anth = normalize(corpus_anth, filter_none = True)
# corpus_add = normalize(corpus_add, filter_none = True)

In [12]:
# print(f"Texte in corpus_anth : {len(corpus_anth)}")
# print(f"Texte in corpus_add  : {len(corpus_add)}")

In [13]:
# test = [x for x in corpus_anth if x.author == 'Geibel, Emanuel' and x.title == 'Schill'][0]

# print(f"id      : {test.id}")
# print(f"source  : {test.source}")
# print(f"author  : {test.author}")
# print(f"title   : {test.title}")
# print(f"content : {test.content}")

In [14]:
# export_corpus_to_txt(corpus_anth, '../resources/corpus_anth_txt/' )
# export_corpus_to_txt(corpus_add, '../resources/corpus_add_txt_exp/' )

# Korpustexte normalisieren

In [15]:
# corpus_anth_norm = normalize(
#     copy.deepcopy(corpus_anth),
#     filter_none = True, 
#     lemmatize = True,
#     lower = True,
#     remove_punctuation = True
# )

In [16]:
# corpus_add_norm = normalize(
#     copy.deepcopy(corpus_add),
#     filter_none = True, 
#     lemmatize = True,
#     lower = True,
#     remove_punctuation = True
# )

In [17]:
# test = [x for x in corpus_anth_norm if x.author == 'Geibel, Emanuel' and x.title == 'Schill'][0]

# print(f"id      : {test.id}")
# print(f"source  : {test.source}")
# print(f"author  : {test.author}")
# print(f"title   : {test.title}")
# print(f"content : {test.content}")

In [18]:
# export_corpus_to_txt(corpus_anth_norm, '../resources/corpus_anth_txt_norm/')
# export_corpus_to_txt(corpus_add_norm, '../resources/corpus_add_txt_norm/')

# Korpustexte re-importieren

In [19]:
corpus_anth = corpus_load_txt("../resources/corpus_anth_txt/*")
corpus_anth_norm = corpus_load_txt("../resources/corpus_anth_txt_norm/*")
corpus_add = normalize(corpus_load_txt("../resources/corpus_add_txt/*"), filter_none = True)
corpus_add_norm = corpus_load_txt("../resources/corpus_add_txt_norm/*")

In [20]:
print(f"Texte in corpus_anth            : {len(corpus_anth)}")
print(f"Texte in corpus_anth_norm       : {len(corpus_anth_norm)}")
print(f"Texte in corpus_add             : {len(corpus_add)}")
print(f"Texte in corpus_add_norm        : {len(corpus_add_norm)}")

Texte in corpus_anth            : 12553
Texte in corpus_anth_norm       : 12553
Texte in corpus_add             : 458
Texte in corpus_add_norm        : 457


# Annotationen einlesen

In [21]:
def read_metadata (path):
    metadata = pd.read_csv(path, sep=';', low_memory=False)
    
    new_header = metadata.iloc[0]
    metadata = metadata[2:]
    metadata.columns = new_header
    
    return metadata

In [22]:
annotations = read_metadata("../resources/more/annotations.csv")

In [23]:
annotations.head(3)

,Quelle/ID,Seite,Band/Kapitel,Autor,GND,Ergänzung,Geburtsjahr,Todesjahr,Titel,Einheitstitel,...,Bewertung der Überlieferung,Bezug auf Geschichtsvorstellungen?,Bewertung der Geschichtsvorstellungen,Verhältnis des Dargestellten zum historischen Wissen,Epistemische Sicherheit,Reim?,Regelmäßiges Metrum?,Verfremdende Sprache?,Distanz,Historische Darstellungsweise?
2,1822.Petri.001,3,1./2.3 – A. Vorbereitung,"Blochmann, Johann Christian Ehrenfried Lebrrecht",http://d-nb.info/116202637,NaN,1777,1840,Klio,Klio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1822.Petri.002,5,1./2.3 – A. Vorbereitung,"Blochmann, Johann Christian Ehrenfried Lebrrecht",http://d-nb.info/116202637,NaN,1777,1840,Kalliope,Kalliope,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1822.Petri.003,6,1./2.3 – A. Vorbereitung,"Kosegarten, Ludwig Gotthard",http://d-nb.info/11898618X,NaN,1758,1818,Ansicht und Würdigung der Geschichte,Ansicht und Würdigung der Geschichte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# meta / Vorbereitung

In [24]:
meta = pd.DataFrame()

In [25]:
# Angabe, welche Spalte in Annotationstabelle welche Kategorie repräsentiert
pos_id = 0
pos_page = 1
pos_authors_names = 3
pos_gnds = 4
pos_ergaenzung = 5
pos_lifetimes_birth = 6
pos_lifetimes_death = 7
pos_title_single = 8
pos_title_unified = 9
pos_text_written = 10
pos_text_published = 11
pos_analysis_start = 14
pos_geschichtslyrik = 14
pos_empirisch = 15
pos_theoretisch = 16
pos_gattung = 17
pos_sprechinstanz_markiert = 18
pos_sprechinstanz_zeitebene = 19
pos_sprechakt = 20
pos_tempus = 21
pos_zeitdominanz = 22
pos_zeitebenen = 23
pos_zeit_fixierbarkeit = 24
pos_time_start = 25
pos_time_end = 26
pos_anachronismus = 27
pos_gegenwartsbezug = 28
pos_grossraum = 29
pos_mittelraum = 30
pos_kleinraum = 31
pos_inhalt_typ = 32
pos_themes = 33
pos_subject = 34
pos_entity = 35
pos_entity_bewertung = 36
pos_themes_bewertung = 37
pos_patriotismus = 38
pos_heroismus = 39
pos_feindbilder = 40
pos_religion = 41
pos_marker_pers = 42
pos_marker_time = 43
pos_marker_place = 44
pos_marker_object = 45
pos_ueberlieferung = 46
pos_ueberlieferung_bewertung = 47
pos_geschichtsvorstellung = 48
pos_geschichtsvorstellung_bewertung = 49
pos_rel_history = 50
pos_sicherheit = 51
pos_reim = 52
pos_metrum = 53
pos_verfremdung = 54
pos_anschaulichkeit = 55
pos_hist_darstellungsweise = 56
pos_analysis_end = 56

# ID, Anthologie, Autor:in, Titel

In [26]:
def get_anthologies (data, RemoveVolume = False):
    anthologies = data.iloc[:,pos_id].values.tolist() # get ids   
    
    # delete .001 etc. (point - 3 digits - any non-word characters - end of word)
    anthologies = [re.sub("\\.[0-9][0-9][0-9][^A-Za-z]*$", "", x) for x in anthologies]

    if RemoveVolume : 
        anthologies = [re.sub("[0-9]$", "", x) for x in anthologies]
        anthologies = [re.sub("\\.$", "", x) for x in anthologies]
    
    return anthologies

In [27]:
def get_anthologies_years (data, always_get_year_of_first_ed = True):
    anthologies = get_anthologies(data)
    anthologies_years = []
        
    anthologies_years_first_ed = [re.findall("[0-9]{4}", x) for x in anthologies]
    anthologies_years_later_ed = [re.findall("\\([0-9]{4}\\)", x) for x in anthologies]
    
    for i in range(len(anthologies)):
        if always_get_year_of_first_ed or anthologies_years_later_ed[i] == []:
            anthologies_years.append(anthologies_years_first_ed[i][0])
        else:
            anthologies_years.append(re.findall("[0-9]{4}", anthologies_years_later_ed[i][0])[0])
    
    anthologies_years = [int(x) for x in anthologies_years]
    
    return anthologies_years

In [28]:
def add_dnb_info (meta):
    dnb_data = pd.read_csv("../resources/more/webscrape_gnd.csv", sep = ',', low_memory = False)
    dnb_data = dnb_data.query("author_gnd_available == True").reset_index(drop = True)
    
    for i, meta_author in enumerate(tqdm(meta['author'])):
        for j, dnb_author in enumerate(dnb_data['author']):
            if dnb_author in meta_author:
                meta.loc[i,"author_gnd"] = dnb_data.iloc[j].author_gnd
                meta.loc[i,"author_gnd_available"] = dnb_data.iloc[j].author_gnd_available
                meta.loc[i,"author_gnd_gender"] = dnb_data.iloc[j].dnb_gender
                meta.loc[i,"author_gnd_birth"] = dnb_data.iloc[j].dnb_birth
                meta.loc[i,"author_gnd_death"] = dnb_data.iloc[j].dnb_death
                meta.loc[i,"author_gnd_countries"] = dnb_data.iloc[j].dnb_countries
                meta.loc[i,"author_gnd_occupations"] = dnb_data.iloc[j].dnb_occupations
        
    return meta

In [29]:
meta['id'] = annotations.iloc[:,pos_id].tolist()

In [30]:
meta['anthology'] = get_anthologies(annotations, RemoveVolume = True)
meta['anthology_with_volume'] = get_anthologies(annotations, RemoveVolume = False)
meta['anthology_year_first_ed'] = get_anthologies_years(annotations, always_get_year_of_first_ed = True)
meta['anthology_year_used_ed'] = get_anthologies_years(annotations, always_get_year_of_first_ed = False)

In [31]:
meta['corpus'] = ['add' if x == '1920.Pinthus' or x == '2022.GeschAddMod' else 'anth' for x in meta['anthology']]

In [32]:
meta['author'] = annotations.iloc[:,pos_authors_names].tolist()

In [33]:
author_gnd = []
for x in annotations.iloc[:,pos_gnds].tolist():
    if '/d-nb.info/' in str(x):
        gnd = re.sub('\\(\\?\\) ', '', x) # delete '(?)'
        gnd = re.sub(' \\[.*\\]', '', gnd) # delete '... [Campe]'
        gnd = re.sub(' +(?=http)', '', gnd) # delete SPACE before ' http'
        author_gnd.append(gnd)
    else:
        author_gnd.append(float('NaN'))
meta['author_gnd'] = author_gnd

In [34]:
author_birth = []
for x in annotations.iloc[:,pos_lifetimes_birth].tolist():
    try:
        author_birth.append(int(x[0:4]))
    except:
        author_birth.append(float('NaN'))
meta['author_birth'] = author_birth

In [35]:
author_death = []
for x in annotations.iloc[:,pos_lifetimes_death].tolist():
    try:
        author_death.append(int(x[0:4]))
    except:
        author_death.append(float('NaN'))
meta['author_death'] = author_death

In [36]:
meta = add_dnb_info(meta)

  0%|          | 0/21303 [00:00<?, ?it/s]

In [37]:
is_historian = []
historian_jobs = [
    'Historiker', 'Altertumswissenschaftler', 'Militärhistoriker', 
    'Genealoge', 'Althistoriker', 'Kirchenhistoriker', 'Geschichtsschreiber',
    'Archivdirektor', 'Chronist', 'Archäologe', 'Archivar', 
] # Literarhistoriker, Kunsthistoriker
for x in meta['author_gnd_occupations']:
    if pd.notna(x) and any([job in x for job in historian_jobs]):
        is_historian.append(1)
    elif pd.notna(x):
        is_historian.append(0)
    else:
        is_historian.append(float('NaN'))
meta['is_historian'] = is_historian

In [38]:
meta['title'] = annotations.iloc[:,pos_title_unified].tolist()
meta['author_title'] = meta['author'] + ' – ' + meta['title']

In [39]:
meta[[
    'corpus', 'id',
    'anthology', 'anthology_with_volume', 'anthology_year_first_ed', 'anthology_year_used_ed',
    'author', 'author_gnd', 'author_birth', 'author_death',
    'author_gnd_countries', 'author_gnd_occupations',
    'title', 'author_title',
]].sample(n=5)

,corpus,id,anthology,anthology_with_volume,anthology_year_first_ed,anthology_year_used_ed,author,author_gnd,author_birth,author_death,author_gnd_countries,author_gnd_occupations,title,author_title
4221,anth,1852.Böttger.4(1862).147,1852.Böttger.4(1862),1852.Böttger.4(1862),1852,1862,"Arndt, Ernst Moritz",http://d-nb.info/118504118,1769.0,1860.0,Deutschland (XA-DE),Schriftsteller + Lyriker + Publizist + Histori...,Die Leipziger Schlacht,"Arndt, Ernst Moritz – Die Leipziger Schlacht"
18556,anth,1926.Wenz.002,1926.Wenz,1926.Wenz,1926,1926,"Uhland, Ludwig",http://d-nb.info/118625063,1787.0,1862.0,Deutschland (XA-DE),Schriftsteller + Literarhistoriker + Hochschul...,Klein Roland,"Uhland, Ludwig – Klein Roland"
7992,anth,1872.Bindewald.2(1875).3.040,1872.Bindewald.2(1875),1872.Bindewald.2(1875).3,1872,1875,"Gerok, Karl",http://d-nb.info/118690876,1815.0,1890.0,Deutschland (XA-DE),Schriftsteller + Evangelischer Theologe,Ritter Bayard in Brescia,"Gerok, Karl – Ritter Bayard in Brescia"
15494,anth,1906/07.Weber.3.038,1906/07.Weber,1906/07.Weber.3,1906,1906,"Görres, Guido",http://d-nb.info/116729228,1805.0,1852.0,Deutschland (XA-DE),Publizist + Schriftsteller,Die Befreiung Wiens,"Görres, Guido – Die Befreiung Wiens"
15205,anth,1903.Stückmann/Ekeris.120,1903.Stückmann/Ekeris,1903.Stückmann/Ekeris,1903,1903,"Wildenbruch, Ernst von",https://d-nb.info/gnd/118771760,1845.0,1909.0,Syrien (XB-SY) + Deutschland (XA-DE) + Libanon...,Schriftsteller + Diplomat + Dramatiker + Juris...,Unser Kaiser Wilhelm,"Wildenbruch, Ernst von – Unser Kaiser Wilhelm"


**id**: ID des Texts, z. B. '1876.Bintz.253'. Die ID liegt in der Regel im Format [Jahr der Anthologie].[Nachname des Anthologie-Herausgebers].[Nummer] vor. Jeder Text erhält eine individuelle ID; auch Dubletten des gleichen Texts erhalten unterschiedliche, individuelle IDs.

**corpus**: Angabe, aus welchem Korpus der Text stammt: 'anth' für Texte aus dem Anthologiekorpus oder 'add' für Texte aus den Ergänzungskorpora

**anthology**: Anthologie, aus der der Text stammt, ohne Angabe des Bandes, z. B. '1876.Bintz', '1881/83.Meyer' oder – im Fall von Texten aus dem Ergänzungskorpus – '2022.GeschAddMod'

**anthology_with_volume**: Anthologie, aus der der Text stammt, inklusive Angabe des Bandes, z. B. '1881/83.Meyer.2' (das '.2' am Ende steht für 'zweiter Band')

**anthology_year_first_ed**: Jahr der ersten Auflage der Anthologie, aus der der Text stammt, z. B. 1872 im Fall eines Texts aus der Anthologie '1872.Bindewald'

**anthology_year_used_ed**: Die Anthologien wurden nach Möglichkeit in erster Auflage eingesehen. Das war aber nicht immer möglich. *anthology_year_used_ed* gibt das Jahr der tatsächlich *genutzten* Auflage der Anthologie an, aus der der Text stammt, z. B. 1875 im Fall eines Texts aus der Anthologie '1872.Bindewald.2(1875)' (in erster Auflage 1872 erschienen, in zweiter Auflage 1875 – die zweite Auflage wurde genutzt) oder 1876 im Fall eines Texts aus der Anthologie '1876.Bintz' (in erster Auflage 1876 erschienen – diese erste Auflage wurde auch genutzt).

**author**: Autor:in des Texts, z. B. 'Fontane, Theodor' oder 'Geibel, Emanuel'

**author_gnd**: GND-Link für die Autor:in des Texts, z. B. 'http://d-nb.info/118534262' im Fall eines Texts von Theodor Fontane oder NaN im Fall eines Texts von einer anonymen Autor:in

**author_birth**: Geburtsjahr der Autor:in des Texts, z. B. 1819 im Fall eines Texts von Theodor Fontane oder NaN im Fall eines Texts von einer anonymen Autor:in

**author_death**: Todesjahr der Autor:in des Texts, z. B. 1898 im Fall eines Texts von Theodor Fontane oder NaN im Fall eines Texts von einer anonymen Autor:in

**[gnd-Daten]**: Verschiedene Angaben zur Autor:in des Texts, die auf Grundlage des DNB-Links aus der GND ausgelesen wurden: 
* *author_gnd_available* (TRUE, falls es einen funktionierenden GND-Link gibt; FALSE, falls nicht)
* *author_gnd_gender* (Geschlecht der Autor:in laut GND: 'männlich', 'weiblich' usw.)
* *author_gnd_birth* (Geburtsjahr der Autor:in laut GND)
* *author_gnd_death* (Todesjahr der Autor:in laut GND)
* *author_gnd_countries* (mit der Autor:in laut GND verknüpfte Länder, z. B. 'Deutschland (XA-DE)' oder 'Deutschland (XA-DE) + Frankreich (XA-FR)')
* *author_gnd_occupations* (mit der Autor:in laut GND verknüpfte Berufe/Tätigkeiten, z. B. 'Schriftsteller' oder 'Schriftsteller + Übersetzer + Historiker')

**title**: Titel des Texts, z. B. 'Der Tag von Hemmingstedt' oder 'Robespierre'. Angegeben wird der *Einheitstitel* des Texts, also nicht unbedingt der in der Anthologie abgedruckte Titel, sondern eine für alle Dubletten des Texts vereinheitlichte Version (also nicht manchmal 'Die Schlacht von Hemmingstedt' und manchmal 'Der Tag von Hemmingstedt', sondern immer einheitlich 'Der Tag von Hemmingstedt' usw.).

**author_title**: Kombination von Titel und Autor, z. B. 'Fontane, Theodor – Der Tag von Hemmingstedt' oder 'Heym, Georg – Robespierre'. Jeder Text ist über *author_title* eindeutig identifizierbar; es gibt niemals zwei (jenseits von Dubletten) unterschiedliche Texte, die dieselbe *author_title*-Angabe erhalten.

# Datierung

## get basic date data

In [40]:
def get_year_search_status (meta):
    written = meta.iloc[:,pos_text_written].tolist()
    published = meta.iloc[:,pos_text_published].tolist()
    
    results = []
    
    for this_written, this_published in zip(written, published):
        if any(char.isdigit() for char in str(this_written)) or any(char.isdigit() for char in str(this_published)):
            results.append('searched_and_found')
        elif '/' in str(this_written) or '/' in str(this_published):
            results.append('searched_but_not_found')
        else:
            results.append('not_searched')
            
    return results    

In [41]:
def get_written_and_published (meta, get_verified_only = False):    
    written = meta.iloc[:,pos_text_written].tolist()
    published = meta.iloc[:,pos_text_published].tolist()
    
    if get_verified_only:
        written = [x if str(x) != 'nan' and 'verified' in str(x) else float('NaN') for x in written]
        published = [x if str(x) != 'nan' and 'verified' in str(x) else float('NaN') for x in published]
     
    # convert to int
    written_int = []
    for x in written:
        try:
            x_clean = re.sub('\\.(.*)', '', re.sub('\\ (.*)', '', str(x)))
            written_int.append(int(x_clean))
        except:
            written_int.append(float('NaN'))

    published_int = []
    for x in published:
        x_clean = re.sub('\\.(.*)', '', re.sub('\\ (.*)', '', str(x)))
        try:
            published_int.append(int(x_clean))
        except:
            published_int.append(float('NaN'))
    
    return [written_int, published_int]

In [42]:
def get_years_gt (written, published):
    years_gt = []
    
    for i in range(len(written)):
        if str(written[i]) != 'nan': years_gt.append(written[i])
        elif str(published[i]) != 'nan': years_gt.append(published[i])
        else: years_gt.append(float('NaN'))
    
    return years_gt

In [43]:
meta['year_search_status'] = get_year_search_status(annotations)

In [44]:
meta['written_gt'] = get_written_and_published(annotations, get_verified_only = False)[0]
meta['published_gt'] = get_written_and_published(annotations, get_verified_only = False)[1]
meta['year_gt'] = get_years_gt(meta.written_gt.tolist(), meta.published_gt.tolist())

In [45]:
print(f"Manuell recherchierte Jahre : {meta['year_gt'].dropna().shape[0]}")

Manuell recherchierte Jahre : 3507


## prepare train/test data

In [46]:
data = (
    meta
    .sort_values(by='year_gt', na_position='last')
    .drop_duplicates(subset="author_title")
    .sort_values(by='author_title')
    .reset_index(drop=True)
    .copy()
)

data = {
    'author_title' : data['author_title'].tolist(),
    'author_birth_year': data['author_birth'].tolist(),
    'author_death_year': data['author_death'].tolist(),
    # 'author_lifespan': (data['author_death']-data['author_birth']).tolist(),
    'first_anth_year': meta.groupby('author_title')['anthology_year_used_ed'].min().tolist(),
    'mean_anth_year': meta.groupby('author_title')['anthology_year_used_ed'].mean().tolist(),
    'last_anth_year': meta.groupby('author_title')['anthology_year_used_ed'].max().tolist(),
    'text_count' : meta.groupby('author_title').size().tolist(),
    'year_gt' : data['year_gt'].tolist()
}
data = pd.DataFrame(data)
data = data.sample(frac=1, random_state=0).reset_index(drop=True)

In [47]:
print(data.shape[0])
data.head()

10446


,author_title,author_birth_year,author_death_year,first_anth_year,mean_anth_year,last_anth_year,text_count,year_gt
0,"Maltiz, Friedrich Franz Apollonius – Schicksal...",1794.0,1857.0,1840,1869.900000,1909,10,NaN
1,"Keiter, Therese – Die Königin",1859.0,1925.0,2022,2022.000000,2022,1,NaN
2,"Kolmar, Gertrud – Marats Antlitz",1894.0,1943.0,2022,2022.000000,2022,1,1934.0
3,"Wildenbruch, Ernst von – Dem Fürsten Bismarck",1845.0,1909.0,1903,1930.166667,1981,6,1890.0
4,"Schollmeyer, Johann Georg – Fürsten-Größe im S...",1768.0,1839.0,1827,1827.000000,1827,1,NaN


In [48]:
possible_features = [
    'author_birth_year',
    'author_death_year', 
    # 'author_lifespan',
    'first_anth_year',
    'mean_anth_year',
    'last_anth_year',
    'text_count',
]

In [49]:
# data_traintest: pick texts where complete data is available
data_traintest = data.dropna(subset=possible_features + ['year_gt'])

print(data_traintest.shape[0])

3395


In [50]:
X = data_traintest[possible_features]
y = data_traintest['year_gt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

print(X_train.shape[0])
print(X_test.shape[0])

2716
679


## Predict

### ages_mean

In [51]:
ages = (data['year_gt']-data['author_birth_year']).dropna().tolist()
ages_mean = np.mean(ages)

In [52]:
meta['year_predict_ages_mean'] = meta['author_birth'] + ages_mean

In [53]:
print(ages_mean)
print(meta.query('year_gt.notna()')['year_predict_ages_mean'].mean())

39.2524609148813
1872.4391506296618


### machine learning

In [54]:
# vanilla
final_feature_combination = possible_features
final_params = RandomForestRegressor(random_state=42).get_params()
X_train_final = X
y_train_final = y

# improved
# final_feature_combination = ['author_birth_year', 'author_death_year', 'first_anth_year', 'last_anth_year']
# final_params = {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
# final_params.update({"random_state":42})
# X_train_final = X_train
# y_train_final = y_train

In [55]:
# create model
random_forest_regressor = RandomForestRegressor(**final_params)
random_forest_regressor.fit(X_train_final[final_feature_combination], y_train_final)

RandomForestRegressor(random_state=42)

In [56]:
# apply model to data
data_predict = data.dropna(subset=final_feature_combination).copy()
data_predict['year_predict'] = random_forest_regressor.predict(data_predict[final_feature_combination])
data_predict = data_predict.reset_index(drop=True)
year_predict_dic = dict(zip(data_predict['author_title'], data_predict['year_predict']))

In [57]:
for i, element in enumerate(meta.iloc):
    this_author_title = element['author_title']

    if this_author_title in data_predict['author_title'].tolist():
        meta.at[i, 'year_predict_rfr'] = year_predict_dic[this_author_title]

## Combine Predictions

In [58]:
earliest_anthology_year_dict = meta.groupby('author_title')['anthology_year_used_ed'].min().to_dict()
year_gt_filled = meta.groupby('author_title')['year_gt'].transform(lambda x: x.ffill().bfill()) 

for i, element in enumerate(meta.iloc):
    final_year = float('NaN')
    
    this_author_title = element['author_title']
    
    this_year_gt = year_gt_filled.iloc[i]
    this_year_predict_rfr = element['year_predict_rfr']
    this_year_predict_ages_mean = element['year_predict_ages_mean']
    
    this_birth_year = element['author_birth']
    this_death_year = element['author_death']
    this_earliest_anthology_year = earliest_anthology_year_dict[this_author_title]
    
    if pd.notna(this_year_gt):
        final_year = this_year_gt
    elif pd.notna(this_year_predict_rfr):
        final_year = round(this_year_predict_rfr)
    elif pd.notna(this_year_predict_ages_mean):
        final_year = round(this_year_predict_ages_mean)

    # check (nicht vor Geburtsdatum, nicht nach Tod, nicht nach erster Anthologie)
    this_min_possible = this_birth_year
    this_max_possible = np.nanmin([this_death_year, this_earliest_anthology_year])
    
    if final_year < this_min_possible:
        final_year = this_min_possible
    elif final_year > this_max_possible:
        final_year = this_max_possible
    
    meta.at[i, 'year'] = final_year

In [59]:
def get_period (meta):
    realism_year_min = 1850
    realism_year_max = 1889
    realism_birth_min = 0
    realism_birth_max = 2000
    
    modernism_year_min = 1890
    modernism_year_max = 1918
    modernism_birth_min = 0
    modernism_birth_max = 2000
    
    periods = []
    
    years = meta["year"].tolist()
    births = meta["author_birth"].tolist()
    
    for this_year, this_birth in zip(years, births):
        if pd.isna(this_year) == False and pd.isna(this_birth) == False:
            if (this_year >= realism_year_min and 
                this_year <= realism_year_max and 
                this_birth >= realism_birth_min and 
                this_birth <= realism_birth_max): 
                periods.append("Realismus")
            elif (this_year >= modernism_year_min and 
                  this_year <= modernism_year_max and 
                  this_birth >= modernism_birth_min and
                  this_birth <= modernism_birth_max): 
                periods.append("Moderne")
            else:
                periods.append("Andere Epoche")
        else:
            periods.append("Unbekannte Epoche")
            
    return periods

In [60]:
meta['decade'] = [(x//10)*10 if pd.isna(x) == False else float('NaN') for x in meta['year']]
meta['period'] = get_period(meta)

## Check

In [61]:
print("year_gt")
print(f"year_gt (count)     : {meta['year_gt'].dropna().shape[0]}")
print(f"year_gt (mean)      : {meta['year_gt'].mean()}")
print(f"ages (mean)         : {(meta['year_gt'].dropna()-meta['author_birth'].dropna()).mean()}")

print("\nages_mean")
print(f"year_predict (mean) : {meta.query('year_gt.notna()')['year_predict_ages_mean'].mean()}")
print(f"Mean Absolute Error : {np.mean(abs(meta['year_predict_ages_mean']-meta['year_gt']))}")
print(f"Mean Squared Error  : {np.mean(abs(meta['year_predict_ages_mean']-meta['year_gt'])**2)}")

print("\nrandom forest")
print(f"year_predict (mean) : {meta.query('year_gt.notna()')['year_predict_rfr'].mean()}")
print(f"Mean Absolute Error : {np.mean(abs(meta['year_predict_rfr']-meta['year_gt']))}")
print(f"Mean Squared Error  : {np.mean(abs(meta['year_predict_rfr']-meta['year_gt'])**2)}")

year_gt
year_gt (count)     : 3507
year_gt (mean)      : 1871.9709153122326
ages (mean)         : 39.2149236531259

ages_mean
year_predict (mean) : 1872.4391506296618
Mean Absolute Error : 9.896605791689172
Mean Squared Error  : 147.65328658988872

random forest
year_predict (mean) : 1872.3506071709696
Mean Absolute Error : 3.1869898798383254
Mean Squared Error  : 27.88486060124036


In [62]:
meta[[
    'id', 'author_title',
    'written_gt', 'published_gt',
    'author_birth', 'author_death',
    'year_gt', 'year_predict_ages_mean', 'year_predict_rfr', 'year',
    'decade', 'period',
]].sample(n=5)

,id,author_title,written_gt,published_gt,author_birth,author_death,year_gt,year_predict_ages_mean,year_predict_rfr,year,decade,period
8581,1876.Bintz.226,Anonym – Eine deutsche Stimme aus dem Jahre 1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unbekannte Epoche
5563,1858.Hocker.021,"Platen, August Graf von – Das Grab im Busento",NaN,NaN,1795.0,1835.0,NaN,1834.252461,1821.180,1820.0,1820.0,Andere Epoche
17610,1917.Götze/Ulbricht.017,"Blomberg, Hugo von – Ein Königswort",NaN,NaN,1820.0,1871.0,NaN,1859.252461,1859.000,1860.0,1860.0,Realismus
2059,1842.Anonym(OestAd).066,"Levitschnigg, Heinrich von – Pauline Fürstin S...",NaN,NaN,1810.0,1862.0,NaN,1849.252461,1839.475,1842.0,1840.0,Andere Epoche
18305,1925.Wehrhan.265,"Lenzberg, Georg – Ein Sang von Lüttich (8. Aug...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unbekannte Epoche


**written_gt**: Manuell recherchiertes Jahr, in dem der Text verfasst wurde, z. B. 1857 oder 1905. Die Daten kommen aus unterschiedlichen Quellen, darunter Werkausgaben, aber auch etwa Websites.

**published_gt**: Manuell recherchiertes Jahr, in dem der Text veröffentlicht wurde, z. B. 1858 oder 1907. Die Daten kommen aus unterschiedlichen Quellen, darunter Werkausgaben, aber auch etwa Websites.

**year_gt**: Manuell recherchiertes Jahr, aus dem der Text stammt, z. B. 1858 oder 1907. Angegeben wird (Priorität 1) das Jahr, in dem der Text verfasst wurde, oder (Priorität 2), falls kein Verfasser-Jahr verfügbar ist, das Jahr, in dem der Text veröffentlicht wurde.

**year**: Jahr, aus dem der Text laut allen verfügbaren Wissensressourcen stammt, z. B. 1858 oder 1907. Angegeben wird (Priorität 1) das manuell recherchierte Jahr, in dem der Text verfasst wurde, oder (Priorität 2), falls kein manuell recherchiertes Verfasser-Jahr verfügbar ist, das manuell recherchierte Jahr, in dem der Text veröffentlicht wurde, oder (Priorität 3), falls kein manuell recherchiertes Veröffentlichungs-Jahr verfügbar ist, der per random forest regressor geschätzte Wert, oder (Priorität 4), falls nicht verfügbar, der per window median smoothed geschätzte Wert.

**decade**: Dekade, aus der der Text stammt. Als Berechnungsgrundlage wird *year* genutzt.

**period**: Epoche, aus der der Text stammt. Unterschieden werden 'Realismus' (Texte zwischen 1850 und 1889), 'Moderne' (Texte zwischen 1890 und 1918), 'Andere Epoche' (Texte vor 1850 oder nach 1918) und 'Unbekannte Epoche' (Texte, für die kein Jahr bekannt ist). Als Berechnungsgrundlage wird *year* genutzt.

# Texte

In [63]:
def get_digitized (corpus, meta):
    digitized_bool = []
    
    ids_digitized = [i.id for i in corpus]
    ids_meta = meta["id"].tolist()
    
    for this_id in ids_meta:
        if this_id in ids_digitized: 
            digitized_bool.append(True)
        else:
            digitized_bool.append(False)
    
    return digitized_bool   

In [64]:
def get_ocr_accuracy(ids):
    ocr_accuracy = []

    anthologies = [re.sub("\\.[0-9][0-9][0-9][^A-Za-z]*$", "", x) for x in ids]

    meta_ocr_anthologies = pd.read_csv("../resources/more/ocr_table.csv", sep=';')
    ocr_anthologies = meta_ocr_anthologies[meta_ocr_anthologies.iloc[:,13].notna()].iloc[:,0].tolist()

    meta_ocr_gt = pd.read_csv("../resources/more/ocr_groundtruth.csv", sep=';')
    ocr_gt_ids = meta_ocr_gt.iloc[:,4].tolist()
    
    add_anthologies = ["1920.Pinthus", "2022.GeschAddMod"]

    for i in range(len(ids)):
        if ids[i] in ocr_gt_ids:
            ocr_accuracy.append(1)
        elif anthologies[i] in add_anthologies:
            ocr_accuracy.append(1)
        elif anthologies[i] in ocr_anthologies:
            this_ocr_error = meta_ocr_anthologies[meta_ocr_anthologies.iloc[:,0] == anthologies[i]].iloc[:,13].tolist()[0]
            this_ocr_error = re.sub(",", ".", this_ocr_error)
            this_ocr_error = float(this_ocr_error)
            ocr_accuracy.append((100 - this_ocr_error) / 100)
        else:
            ocr_accuracy.append(float('NaN'))
    
    return ocr_accuracy

In [65]:
def get_ocr_column_error (ids):
    ocr_column_error = []
    
    anthologies = [re.sub("\\.[0-9][0-9][0-9][^A-Za-z]*$", "", x) for x in ids]
    
    meta_ocr_anthologies = pd.read_csv("../resources/more/ocr_table.csv", sep=';')
    ocr_anthologies = meta_ocr_anthologies[meta_ocr_anthologies.iloc[:,1].notna()].iloc[:,0].tolist()
    
    add_anthologies = ["1920.Pinthus", "2022.GeschAddMod"]
    
    for i in range(len(ids)):
        if anthologies[i] in ocr_anthologies:
            this_ocr_column_error = meta_ocr_anthologies[meta_ocr_anthologies.iloc[:,0] == anthologies[i]].iloc[:,1].tolist()[0]
            if 'korrigiert' in this_ocr_column_error: 
                this_ocr_column_error = 0
            else:
                this_ocr_column_error = float('0.' + this_ocr_column_error[0])
            ocr_column_error.append(this_ocr_column_error)
        elif anthologies[i] in add_anthologies:
            ocr_column_error.append(0)
        else:
            ocr_column_error.append(float('NaN'))
    
    return ocr_column_error

In [66]:
def transform_ids_to_bestocr_ids(ids, meta):
    # Precompute lookups for `author_title` and group by it
    grouped_meta = meta.groupby("author_title")
    
    # Prepare a dictionary to store best IDs per author_title
    best_ids = {}
    
    for author_title, group in grouped_meta:
        # Filter and sort the group once
        sorted_group = group.sort_values(by="ocr_accuracy", ascending=False)
        digitized = sorted_group[sorted_group["digitized"] == True]
        digitized_no_col_errors = digitized[digitized["ocr_column_error"] == 0]
        
        # Select the best ID based on the filtering
        if not digitized_no_col_errors.empty:
            best_ids[author_title] = digitized_no_col_errors.iloc[0]["id"]
        elif not digitized.empty:
            best_ids[author_title] = digitized.iloc[0]["id"]
        else:
            best_ids[author_title] = sorted_group.iloc[0]["id"]
    
    # Map each ID to its best corresponding ID
    updated_ids = [best_ids[meta.loc[meta["id"] == this_id, "author_title"].iloc[0]] for this_id in tqdm(ids)]
    
    return updated_ids

In [67]:
def get_texts_by_id(ids, corpus, return_text_object=False):
    # Create a lookup dictionary for corpus texts by their IDs
    corpus_dict = {text.id: text for text in corpus}
    
    texts = []
    for i in ids:
        if i in corpus_dict:
            this_text = corpus_dict[i]
            if return_text_object:
                texts.append(this_text)
            else:
                texts.append(this_text.content)
        else:
            texts.append(float('NaN'))
    
    return texts


In [68]:
meta['digitized'] = get_digitized(corpus_anth + corpus_add, meta)

In [69]:
meta['ocr_accuracy'] = get_ocr_accuracy(meta['id'].tolist())
meta['ocr_column_error'] = get_ocr_column_error(meta['id'].tolist())

In [70]:
meta['id_bestocr'] = transform_ids_to_bestocr_ids(meta['id'].tolist(), meta)

  0%|          | 0/21303 [00:00<?, ?it/s]

In [71]:
meta['ocr_accuracy_bestocr'] = get_ocr_accuracy(meta['id_bestocr'].tolist())
meta['ocr_column_error_bestocr'] = get_ocr_column_error(meta['id_bestocr'].tolist())

In [72]:
meta['text'] = get_texts_by_id(
    ids = meta['id'].tolist(), 
    corpus = corpus_anth + corpus_add
)
meta['text_bestocr'] = get_texts_by_id(
    ids = meta['id_bestocr'].tolist(), 
    corpus = corpus_anth + corpus_add
)

In [73]:
meta['text_normalized'] = get_texts_by_id(
    ids = meta['id'].tolist(), 
    corpus = corpus_anth_norm + corpus_add_norm
)
meta['text_normalized_bestocr'] = get_texts_by_id(
    ids = meta['id_bestocr'].tolist(), 
    corpus = corpus_anth_norm + corpus_add_norm
)

In [74]:
meta[[
    'id', 'author_title',
    'digitized',
    'ocr_accuracy', 'ocr_column_error',
    'id_bestocr',
    'ocr_accuracy_bestocr', 'ocr_column_error_bestocr',
    'text', 'text_bestocr', 
    'text_normalized', 'text_normalized_bestocr',
]].sample(n=5)

,id,author_title,digitized,ocr_accuracy,ocr_column_error,id_bestocr,ocr_accuracy_bestocr,ocr_column_error_bestocr,text,text_bestocr,text_normalized,text_normalized_bestocr
5941,1858.Hocker.382,"Bandemer, Susanne von – An Ramler",False,NaN,NaN,1858.Hocker.382,NaN,NaN,NaN,NaN,NaN,NaN
13614,1892.Dietlein.007,"Pfizer, Gustav – Alarichs Grab",True,0.999239,0.5,1892.Dietlein.007,0.999239,0.5,"[Was ist dem kühnen Volke wider- Die Schätze, ...","[Was ist dem kühnen Volke wider- Die Schätze, ...",[was sein der kühn volk wider der schatz vom ...,[was sein der kühn volk wider der schatz vom ...
16930,1913.Schrutz.1.014,"Uhland, Ludwig – König Karls Meerfahrt",True,0.995221,0.0,1892/93.Tetzner.1.089,0.999902,0.0,"[Der Konig Karl fuhr über Meer, Mit seinen zwo...","[Der König Karl fuhr über Meer, Mit seinen zwö...","[der konig karl fahren über meer, mit seinen z...","[der könig karl fahren über meer, mit seinen z..."
11714,1886.Bliedner.080,"Gerok, Karl – Luther auf der Veste Koburg",True,0.998401,0.5,1904.Linde.054,0.999965,0.0,[ls Luther auf der Koburg 5. Wie Moses auf dem...,"[Als Luther auf der Koburg lag,, Von Acht und ...",[ls luther auf der koburg 5 wie moses auf der ...,"[als luther auf der koburg liegen , von acht u..."
16855,1912.Werner.528,"Steller, Konrad Gustav – Das Weberhaus von Don...",True,0.996168,0.0,1912.Werner.528,0.996168,0.0,"[Durch den graunden Morgen reitet, Über Felder...","[Durch den graunden Morgen reitet, Über Felder...","[durch der graunden morgen reiten, über feld ...","[durch der graunden morgen reiten, über feld ..."


**digitized**: Angabe (TRUE/FALSE), ob für den Text ein Digitalisat vorliegt

**ocr_accuracy**: Angabe (falls für den Text ein Digitalisat vorliegt), wie hoch die Accuracy der OCR-Texterkennung ist, z. B. 0.9988

**ocr_column_error**: Angabe (falls für den Text ein Digitalisat vorliegt), mit welcher Wahrscheinlichkeit in dem Digitalisat Verse in falscher Reihenfolge angeordnet sind, z. B. 0 (= es gibt keine derartigen Fehler) oder 0.5 (jeder zweite Text ist von derartigen Fehlern betroffen). Das kann vorkommen, wenn eine Anthologie nicht durchgängig ein- oder mehrspaltig, sondern in variierender Spaltenzahl formatiert ist und dies bei der Texterkennung nicht einberechnet wurde. Die Wahrscheinlichkeiten (außer 0 = es gibt keine derartigen Fehler) wurden für jede Anthologie manuell geschätzt.

**id_bestocr**: Angabe der ID des Texts, der für den vorliegenden Text das qualitativ beste Digitalisat bereitstellen kann. Falls für den Text keine Dubletten im Korpus existieren und er insofern nur 1x digitalisiert wurde, ist *id_bestocr* identisch mit der *id* dieses Texts. Ansonsten wird unter allen Dubletten/Digitalisaten für den Text die beste Version ausgewählt, und zwar nach folgenden Prioritäten: (1) Digitalisat ohne Fehler bei der Versreihenfolge und dann mit möglichst hoher OCR-Accuracy; oder, falls jedes Digitalisat eine gewisse Wahrscheinlichkeit für Fehler bei der Versreihenfolge aufweist, (2) Digitalisat mit möglichst hoher OCR-Accuracy.

**ocr_accuracy_bestocr**: Angabe, wie hoch die Accuracy der OCR-Texterkennung für den Text mit der unter *id_bestocr* angegebenen ID ist, z. B. 0.9998

**ocr_column_error_bestocr**: Angabe, mit welcher Wahrscheinlichkeit in dem Digitalisat des Texts mit der unter *id_bestocr* angegebenen ID Verse in falscher Reihenfolge angeordnet sind.

**text**: Text als Liste von Strings, wobei jeder String einen Vers repräsentiert

**text_bestocr** Text des Texts mit der unter *id_bestocr* angegebenen ID, wobei jeder String einen Vers repräsentiert

**text_normalized** Text als Liste von Strings, wobei jeder String einen Vers repräsentiert. Der Text wurde normalisiert (Lemmatisierung, durchgängige Kleinschreibung, Entfernung von Satzzeichen).

**text_normalized_bestocr** Text des Texts mit der unter *id_bestocr* angegebenen ID als Liste von Strings, wobei jeder String einen Vers repräsentiert. Der Text wurde normalisiert (Lemmatisierung, durchgängige Kleinschreibung, Entfernung von Satzzeichen).

# Annotationen

In [75]:
def get_titles_annotated (data, data_type = "pos_unique", min_annotated = 20):    
    titles_with_authors = data.iloc[:,pos_authors_names] + ' – ' + data.iloc[:,pos_title_unified]

    annotated_pos_unique = []
    annotated_pos_all = []
    annotated_titles_with_authors = []

    for i in range(len(data)):
        if data.shape[1] - data.iloc[i].isna().sum() - 7 >= min_annotated:
            annotated_pos_unique.append(True)
        else:
            annotated_pos_unique.append(False)

    annotated_titles_with_authors = np.array(titles_with_authors)[annotated_pos_unique].tolist()
    
    for element in titles_with_authors:
        if element in annotated_titles_with_authors:
            annotated_pos_all.append(True)
        else:
            annotated_pos_all.append(False)
    
    if data_type == "pos_unique":
        return annotated_pos_unique 
    if data_type == "pos_all":
        return annotated_pos_all 
    if data_type == "titles":
        return annotated_titles_with_authors
    
meta['annotated'] = get_titles_annotated(annotations, data_type = 'pos_unique')

In [76]:
#01 Geschichtslyrik
def get_geschichtslyrik (data, data_type = "value"):
    geschichtslyrik = data.iloc[:,pos_geschichtslyrik].tolist()
    
    geschichtslyrik_value = [re.sub("\\[(.*)\\]$", "", x) if pd.isna(x) == False else x for x in geschichtslyrik]
    geschichtslyrik_value = [float(x) for x in geschichtslyrik_value]
    
    geschichtslyrik_comment = []
    for x in geschichtslyrik:
        if pd.isna(x):
            geschichtslyrik_comment.append(x)
        else:
            if re.search("\\[(.*)\\]$", x) != None:
                geschichtslyrik_comment.append(re.search("\\[(.*)\\]$", x).group(1))
    
    if data_type == "value": return (geschichtslyrik_value)
    if data_type == "comment": return (geschichtslyrik_comment)
    
meta['geschichtslyrik'] = get_geschichtslyrik(annotations, data_type = 'value')

In [77]:
#02 Empirische Ausrichtung
empirisch = [float(x) for x in annotations.iloc[:,pos_empirisch]]
empirisch = [1 if x > 0 else x for x in empirisch]

meta['empirisch'] = empirisch

In [78]:
#03 Geschichtstheoretische Ausrichtung
theoretisch = [float(x) for x in annotations.iloc[:,pos_theoretisch]]
theoretisch = [1 if x > 0 else x for x in theoretisch]

meta['theoretisch'] = theoretisch

In [79]:
#04 Gattung
moegliche_gattungen = ['Ballade', 'Lied', 'Rollengedicht', 'Sonett', 'Denkmal-/Ruinenpoesie']

gattung = []
for x in annotations.iloc[:,pos_gattung]:
    if pd.isna(x):
        gattung.append(x)
    else:
        x = re.sub(' \\[.*?\\]', '', x)
        x = x.split(' + ')
        x = sorted(x)
        x = [y for y in x if y in moegliche_gattungen]
        x = ' + '.join(x)
        x = float('NaN') if x == '' else x
        gattung.append(x)

meta['gattung'] = gattung

In [80]:
#05 Markiertheit der Sprechinstanz
meta['sprechinstanz_markiert'] = [float(x) for x in annotations.iloc[:,pos_sprechinstanz_markiert]]

In [81]:
#06 Zeitliche Position der Sprechinstanz
sprechinstanz_in_vergangenheit = [float(x) for x in annotations.iloc[:,pos_sprechinstanz_zeitebene]]
sprechinstanz_in_vergangenheit = [0 if x == 2 else x for x in sprechinstanz_in_vergangenheit]

meta['sprechinstanz_in_vergangenheit'] = sprechinstanz_in_vergangenheit

In [82]:
#07 Sprechakte
sprechakte_dict = {'Lobpreisen' : 'Behaupten', 
                   'Prophezeien' : 'Behaupten', 
                   'Wünschen' : 'Auffordern',
                   'Verfluchen' : 'Behaupten',
                   'Reflektieren' : 'Beschreiben',
                   'Reflektieren [Figur]' : 'Beschreiben',
                   'Imaginieren' : 'Beschreiben',
                  }

sprechakte = []
for x in annotations.iloc[:,pos_sprechakt]:
    if pd.isna(x):
        sprechakte.append(x)
    else:
        x = x.split(' + ')
        x = set([sprechakte_dict.get(y) if y in sprechakte_dict else y for y in x])
        x = sorted(x)
        x = ' + '.join(x)
        sprechakte.append(x)

meta['sprechakte'] = sprechakte

In [83]:
#08 Tempora
tempus = []
for x in annotations.iloc[:,pos_tempus]:
    if pd.isna(x):
        tempus.append(x)
    else:
        x = x.split(' + ')
        x = sorted(x)
        x = ' + '.join(x)
        tempus.append(x)

meta['tempus'] = tempus

In [84]:
#09 Konkretheit
meta['konkretheit'] = [float(x) for x in annotations.iloc[:,pos_anschaulichkeit]]

In [85]:
#10 Wissen der Sprechinstanz
wissen_dic = {0:float('NaN'),1:1,2:-1,3:0}
wissen = [float(x) for x in annotations.iloc[:,pos_sicherheit]]
wissen = [wissen_dic.get(x) if x in wissen_dic else x for x in wissen]

meta['wissen'] = wissen

In [86]:
#11 Zeitdominanz
vergangenheitsdominant = [float(x) for x in annotations.iloc[:,pos_zeitdominanz]]
vergangenheitsdominant = [0 if x == 2 else x for x in vergangenheitsdominant]
vergangenheitsdominant = [0.5 if x == 3 else x for x in vergangenheitsdominant]

meta['vergangenheitsdominant'] = vergangenheitsdominant

In [87]:
#12 Anzahl Zeitebenen
meta['zeitebenen'] = [float(x) for x in annotations.iloc[:,pos_zeitebenen]]

In [88]:
#13 Fixierbarkeit der dominanten Zeitebene
fixierbarkeit = [float(x) for x in annotations.iloc[:,pos_zeit_fixierbarkeit]]
fixierbarkeit = [0 if x == 2 else x for x in fixierbarkeit]

meta['fixierbarkeit'] = fixierbarkeit

In [89]:
#14 Beginn der dominanten Zeitebene
meta['beginn'] = [float(x) for x in annotations.iloc[:,pos_time_start]]

In [90]:
#15 Ende der dominanten Zeitebene
meta['ende'] = [float(x) for x in annotations.iloc[:,pos_time_end]]

In [91]:
#16 Anachronismen
anachronismus = []
for x in annotations.iloc[:,pos_anachronismus]:
    if pd.isna(x):
        anachronismus.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        anachronismus.append(x)

meta['anachronismus'] = anachronismus

In [92]:
#17 Gegenwartsbezug
meta['gegenwartsbezug'] = [float(x) for x in annotations.iloc[:,pos_gegenwartsbezug]]

In [93]:
#18 Großraum
grossraum = []
for x in annotations.iloc[:,pos_grossraum]:
    if pd.isna(x):
        grossraum.append(x)
    else:
        x = x.split(' + ')
        x = sorted(x)
        x = ' + '.join(x)
        grossraum.append(x)

meta['grossraum'] = grossraum

In [94]:
#19 Mittelraum
mittelraum = []
for x in annotations.iloc[:,pos_mittelraum]:
    if pd.isna(x):
        mittelraum.append(x)
    else:
        x = x.split(' + ')
        x = sorted(x)
        x = ' + '.join(x)
        x = re.sub(' \\[.*?\\]', '', x)
        mittelraum.append(x)

meta['mittelraum'] = mittelraum

In [95]:
#20 Kleinraum
kleinraum = []
for x in annotations.iloc[:,pos_kleinraum]:
    if pd.isna(x):
        kleinraum.append(x)
    else:
        x = x.split(' + ')
        x = sorted(x)
        x = ' + '.join(x)
        x = re.sub(' \\[.*?\\]', '', x)
        kleinraum.append(x)

meta['kleinraum'] = kleinraum

In [96]:
#21 Inhaltstyp
inhaltstyp_dict = {'Ereignismenge' : 'Ereignis', 
                   'Prozess' : 'Ereignis', 
                   'Situation' : 'Zustand',
                   'Person' : 'Zustand',
                   'Objekt' : 'Zustand',
                  }

inhaltstyp = []
for x in annotations.iloc[:,pos_inhalt_typ]:
    if pd.isna(x):
        inhaltstyp.append(x)
    else:
        x = x.split(' + ')
        x = set([inhaltstyp_dict.get(y) if y in inhaltstyp_dict else y for y in x])
        x = sorted(x)
        x = ' + '.join(x)
        inhaltstyp.append(x)

meta['inhaltstyp'] = inhaltstyp

In [97]:
#22 Stoffgebiet
stoffgebiet_dict = {'Krieg' : 'Militär/Krieg'}

stoffgebiet = []
for x in annotations.iloc[:,pos_themes]:
    if pd.isna(x):
        stoffgebiet.append(x)
    else:
        x = re.sub(' \\[.*?\\]', '', x)
        x = x.split(' + ')
        x = [stoffgebiet_dict.get(y) if y in stoffgebiet_dict else y for y in x]
        x = ' + '.join(x)
        stoffgebiet.append(x)

meta['stoffgebiet'] = stoffgebiet

In [98]:
#23 Bewertung Stoffgebiet
meta['stoffgebiet_bewertung'] = annotations.iloc[:,pos_themes_bewertung].tolist()

In [99]:
#24 Entitäten
entity_simple = []
for x in annotations.iloc[:,pos_entity]:
    if pd.isna(x):
        entity_simple.append(x)
    else:
        x = re.sub(' \\[.*?\\]', '', x)
        x = re.sub('5', '4', x)
        entity_simple.append(x)

meta['entity_full'] = annotations.iloc[:,pos_entity].tolist()
meta['entity_simple'] = entity_simple

In [100]:
#25 Bewertung Entitäten
meta['entity_bewertung'] = annotations.iloc[:,pos_entity_bewertung].tolist()

In [101]:
#26 Nationalismus
nationalismus = []
for x in annotations.iloc[:,pos_patriotismus]:
    if pd.isna(x):
        nationalismus.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = 1 if x == 3 else x
        x = 0 if x == 2 else x
        nationalismus.append(x)
        
meta['nationalismus'] = nationalismus

In [102]:
#27 Heroismus
heroismus = []
for x in annotations.iloc[:,pos_heroismus]:
    if pd.isna(x):
        heroismus.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = 1 if x == 3 else x
        x = 0 if x == 2 else x
        heroismus.append(x)
        
meta['heroismus'] = heroismus

In [103]:
#28 Religiosität
religiositaet = []
for x in annotations.iloc[:,pos_religion]:
    if pd.isna(x):
        religiositaet.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = 1 if x == 3 else x
        x = 0 if x == 2 else x
        religiositaet.append(x)
        
meta['religiositaet'] = religiositaet

In [104]:
#29 Geschichtsmarker Person
marker_dict = {0 : '/', 1 : 'Titel', 2 : 'Text', 3 : 'Titel + Text'}

marker_person = []
for x in annotations.iloc[:,pos_marker_pers]:
    if pd.isna(x):
        marker_person.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = marker_dict.get(x) if x in marker_dict else x
        marker_person.append(x)
        
meta['marker_person'] = marker_person

In [105]:
#30 Geschichtsmarker Zeit
marker_dict = {0 : '/', 1 : 'Titel', 2 : 'Text', 3 : 'Titel + Text'}

marker_zeit = []
for x in annotations.iloc[:,pos_marker_time]:
    if pd.isna(x):
        marker_zeit.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = marker_dict.get(x) if x in marker_dict else x
        marker_zeit.append(x)
        
meta['marker_zeit'] = marker_zeit

In [106]:
#31 Geschichtsmarker Ort
marker_dict = {0 : '/', 1 : 'Titel', 2 : 'Text', 3 : 'Titel + Text'}

marker_ort = []
for x in annotations.iloc[:,pos_marker_place]:
    if pd.isna(x):
        marker_ort.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = marker_dict.get(x) if x in marker_dict else x
        marker_ort.append(x)
        
meta['marker_ort'] = marker_ort

In [107]:
#32 Geschichtsmarker Objekt
marker_dict = {0 : '/', 1 : 'Titel', 2 : 'Text', 3 : 'Titel + Text'}

marker_objekt = []
for x in annotations.iloc[:,pos_marker_object]:
    if pd.isna(x):
        marker_objekt.append(x)
    else:
        x = str(x)[0]
        x = int(x)
        x = marker_dict.get(x) if x in marker_dict else x
        marker_objekt.append(x)
        
meta['marker_objekt'] = marker_objekt

In [108]:
#33 Überlieferung
ueberlieferung = []
for x in annotations.iloc[:,pos_ueberlieferung]:
    if pd.isna(x):
        ueberlieferung.append(x)
    elif str(x) == '0':
        ueberlieferung.append(0)
    else:
        ueberlieferung.append(1)

meta['ueberlieferung'] = ueberlieferung

In [109]:
#34 Bewertung Überlieferung
value_dic = {0 : 'neutral', 1 : 'positiv', 2 : 'negativ', 3 : 'ambivalent'}
    
ueberlieferung_bewertung = []
for x in annotations.iloc[:,pos_ueberlieferung_bewertung]:
    if pd.isna(x):
        ueberlieferung_bewertung.append(x)
    else:
        x = x.split(" + ")
        x = [int(y) for y in x]
        x = stats.mode(x, keepdims = True)[0][0]
        x = value_dic.get(x) if x in value_dic else x
        ueberlieferung_bewertung.append(x)

meta['ueberlieferung_bewertung'] = ueberlieferung_bewertung

In [110]:
#35 Geschichtsauffassungen
geschichtsauffassung = []
for x in annotations.iloc[:,pos_geschichtsvorstellung]:
    if pd.isna(x):
        geschichtsauffassung.append(x)
    elif str(x) == '0':
        geschichtsauffassung.append(0)
    else:
        geschichtsauffassung.append(1)

meta['geschichtsauffassung'] = geschichtsauffassung

In [111]:
#36 Bewertung Geschichtsauffassungen
value_dic = {0 : 'neutral', 1 : 'positiv', 2 : 'negativ', 3 : 'ambivalent'}
    
geschichtsauffassung_bewertung = []
for x in annotations.iloc[:,pos_geschichtsvorstellung_bewertung]:
    if pd.isna(x):
        geschichtsauffassung_bewertung.append(x)
    else:
        x = x.split(" + ")
        x = [int(y) for y in x]
        x = stats.mode(x, keepdims = True)[0][0]
        x = value_dic.get(x) if x in value_dic else x
        geschichtsauffassung_bewertung.append(x)

meta['geschichtsauffassung_bewertung'] = geschichtsauffassung_bewertung

In [112]:
#37 Verhältnis zum historischen Wissen
wissen_dic = {0 : 'übereinstimmend', 1 : 'ergänzend', 2 : 'abweichend_übernatürlich', 3 : 'abweichend_natürlich'}
    
verhaeltnis_wissen = []
for x in annotations.iloc[:,pos_rel_history]:
    if pd.isna(x):
        verhaeltnis_wissen.append(x)
    else:
        x = re.sub(' \\[.*?\\]', '', x)
        x = x.split(" + ")
        x = [int(y) for y in x]
        x = sorted(x)
        if len(x) > 1:
            x = [y for y in x if y == 2 or y == 3]
        x = [wissen_dic.get(y) if y in wissen_dic else y for y in x]
        x = ' + '.join(x)
        verhaeltnis_wissen.append(x)

meta['verhaeltnis_wissen'] = verhaeltnis_wissen

In [113]:
#38 Reim
meta['reim'] = [float(x) for x in annotations.iloc[:,pos_reim]]

In [114]:
#39 Metrum
meta['metrum'] = [float(x) for x in annotations.iloc[:,pos_metrum]]

In [115]:
#40 Verfremdende Sprache
meta['verfremdung'] = [float(x) for x in annotations.iloc[:,pos_verfremdung]]

In [116]:
meta.query("annotated")[[
    'id', 'author_title',
    'annotated',
    'geschichtslyrik', 'gattung', 'sprechakte', 'stoffgebiet', 'reim',
]].sample(n=5)

,id,author_title,annotated,geschichtslyrik,gattung,sprechakte,stoffgebiet,reim
7840,1872.Bindewald.2(1875).2.170,"Dörr, Adolf – Kaiser Friedrichs II. Tod",True,1.0,Ballade,Beschreiben + Erzählen,Tod,1.0
18424,1926.Beck.078,"Meyer, Conrad Ferdinand – Huttens letzte Tage....",True,1.0,Rollengedicht,Erzählen,Religion + Militär/Krieg,1.0
19183,1938.Plenzat.060,"Jensen, Wilhelm – Zwölfnacht",True,0.0,Ballade,Erzählen,Religion + Armut,1.0
18436,1926.Beck.090,"Löns, Hermann – Der Bohrturm",True,0.0,NaN,Behaupten + Beschreiben,Moderne,1.0
12893,1891.Brümmer.042,"Hertz, Wilhelm – Alboin und Rosamunde",True,1.0,Ballade,Erzählen,Liebe + Tod,1.0


**annotated**: Angabe (TRUE/FALSE), ob der Text annotiert wurde

**[Annotationskategorien]**: Alle Annotationskategorien samt Annotationsergebnissen, z. B. 'Erzählen' im Fall der Kategorie *Sprechakte* oder 'Präsens' im Fall der Kategorie *Tempus*.

# Dubletten auffüllen

Für viele Daten, die für Text A erhoben wurden, gilt, dass sie sich auf die Texte B, C, D, ... übertragen lassen, sofern es sich bei B, C, D, ... um Dubletten von A handelt (wenn für den annnotierten Text 'Fontane, Theodor – Der Tag von Hemmingstedt' in der Anthologie Y1 gilt, dass er eine Ballade ist, gilt das auch für die Dubletten dieses Texts in den Anthologien Y2, Y3, Y4, ...). Diese Übertragung wird hier geleistet; alle Texte mit dem gleichen Wert bei *author_title* erhalten die gleichen Einträge. Dabei werden einige Kategorien, die für jeden Text (auch für jede Dublette) individuell bleiben sollen, ausgenommen, z. B. *id* oder *anthology*.

In [117]:
meta.query("author.str.contains('Münchhausen') & title.str.contains('Die Trommel des Ziska')")[[
    'author_title',
    'id',
    'ocr_accuracy',
    'digitized',
    'text',
    'text_bestocr',
    'published_gt',
    'annotated',
    'geschichtslyrik',
    'gattung',
    'sprechakte',
    'stoffgebiet',
    'stoffgebiet_bewertung',
    'grossraum'
]].head()

,author_title,id,ocr_accuracy,digitized,text,text_bestocr,published_gt,annotated,geschichtslyrik,gattung,sprechakte,stoffgebiet,stoffgebiet_bewertung,grossraum
15467,"Münchhausen, Börries von – Die Trommel des Ziska",1906/07.Weber.3.011,1.000000,True,"[Weit in Böhmen herum, herum, Klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
15978,"Münchhausen, Börries von – Die Trommel des Ziska",1908.Berg.059,0.995497,True,"[Weit in Böhmen herum, herum,, Klopfen die Tro...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa
16252,"Münchhausen, Börries von – Die Trommel des Ziska",1912.Harten/Henniger.1.031,0.998210,True,"[eit in Böhmen herum, herum, Klopfen die Tromm...","[Weit in Böhmen herum, herum, Klopfen die Trom...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
16585,"Münchhausen, Börries von – Die Trommel des Ziska",1912.Werner.261,0.996168,True,"[Weit in Böhmen herum, herum, Klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
17239,"Münchhausen, Börries von – Die Trommel des Ziska",1915.Eggert-Windegg.139,0.999236,True,"[Weit in Böhmen herum, herum, klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
def fill_metadata(data, exceptions=["id", "anthology", "digitized"]):
    # Determine categories to fill
    fill_categories = [x for x in data.columns if x not in exceptions]
    
    # Filter `author_title` groups that occur more than once
    author_title_counts = data['author_title'].value_counts()
    author_titles_to_process = author_title_counts[author_title_counts > 1].index
    
    # Process each fill category
    for category in tqdm(fill_categories, desc="Processing categories"):
        def fill_group(values):
            if values.isnull().all():
                return float('NaN')  # All NaN
            elif all(isinstance(x, bool) for x in values.dropna()):
                return True if True in values.values else False  # Logical OR for booleans
            else:
                return values.dropna().max()  # Highest non-NaN value
        
        # Apply the fill logic group-wise
        filled_values = data[data['author_title'].isin(author_titles_to_process)].groupby('author_title')[category].transform(fill_group)
        data.loc[data['author_title'].isin(author_titles_to_process), category] = filled_values
    
    return data


In [119]:
# meta_nofill = copy.deepcopy(meta)
# meta_nofill.to_csv(r"../02_resources/csv_meta_nofill.csv", index = False, header = True)

In [120]:
meta = fill_metadata(
    meta.copy(), 
    exceptions = [
        'corpus',
        'id',
        'id_bestocr',
        'digitized',
        'ocr_accuracy',
        'ocr_accuracy_bestocr',
        'ocr_column_error',
        'ocr_column_error_bestocr',
        'text',
        'text_bestocr',
        'text_normalized',
        'text_normalized_bestocr',
        'anthology',
        'anthology_with_volume',
        'anthology_year_first_ed',
        'anthology_year_used_ed',
    ]
)

Processing categories:   0%|          | 0/64 [00:00<?, ?it/s]

In [121]:
meta.query("author.str.contains('Münchhausen') & title.str.contains('Die Trommel des Ziska')")[[
    'author_title',
    'id',
    'ocr_accuracy',
    'digitized',
    'text',
    'text_bestocr',
    'published_gt',
    'annotated',
    'geschichtslyrik',
    'gattung',
    'sprechakte',
    'stoffgebiet',
    'stoffgebiet_bewertung',
    'grossraum'
]].head()

,author_title,id,ocr_accuracy,digitized,text,text_bestocr,published_gt,annotated,geschichtslyrik,gattung,sprechakte,stoffgebiet,stoffgebiet_bewertung,grossraum
15467,"Münchhausen, Börries von – Die Trommel des Ziska",1906/07.Weber.3.011,1.000000,True,"[Weit in Böhmen herum, herum, Klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa
15978,"Münchhausen, Börries von – Die Trommel des Ziska",1908.Berg.059,0.995497,True,"[Weit in Böhmen herum, herum,, Klopfen die Tro...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa
16252,"Münchhausen, Börries von – Die Trommel des Ziska",1912.Harten/Henniger.1.031,0.998210,True,"[eit in Böhmen herum, herum, Klopfen die Tromm...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa
16585,"Münchhausen, Börries von – Die Trommel des Ziska",1912.Werner.261,0.996168,True,"[Weit in Böhmen herum, herum, Klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa
17239,"Münchhausen, Börries von – Die Trommel des Ziska",1915.Eggert-Windegg.139,0.999236,True,"[Weit in Böhmen herum, herum, klopfen die Trom...","[Weit in Böhmen herum, herum, Klopfen die Trom...",1903.0,True,1.0,Ballade,Erzählen,Militär/Krieg,2,Europa


# Check

In [122]:
modcanon_authors = ['Hofmannsthal, Hugo von', 'Rilke, Rainer Maria', 'George, Stefan', 'Heym, Georg']
muench_authors = ['Münchhausen, Börries von', 'Miegel, Agnes', 'Strauß und Torney, Lulu von']

In [123]:
# Fehlende Datierungen?
pd.set_option('display.width', 1000)

texts_to_date = (
    meta
    .query("1850 <= year <= 1918")
    .query("corpus == 'anth' or author in @modcanon_authors or author in @muench_authors")
    .query("year_search_status == 'not_searched'")
    .drop_duplicates(subset='author_title')
    .sort_values(by = 'year')
)

if texts_to_date.shape[0] == 0:
    print('Keine fehlenden Datierungen')
else:
    print('Fehlende Datierungen:\n')
    print(texts_to_date[['id', 'author', 'title', 'year', 'year_gt', 'corpus']])

# texts_to_date[['id', 'author', 'title', 'year', 'year_gt', 'corpus', 'text_bestocr']].to_csv("fehlende_datierungen.csv")

test_ids = ['1892/93.Tetzner.1.067', '1891.Brümmer.521']
test_results = meta.query("id.isin(@test_ids)")
print("\nTests:")
print(test_results[['id', 'author', 'title', 'year', 'year_gt', 'year_predict_ages_mean', 'year_predict_rfr']])

Keine fehlenden Datierungen

Tests:
                          id          author                         title    year  year_gt  year_predict_ages_mean  year_predict_rfr
13395       1891.Brümmer.521  Döring, Moritz  Die weiße Kuh von Courcelles  1850.0      NaN             1837.252461          1849.705
13789  1892/93.Tetzner.1.067     Dahn, Felix                      Gotenzug  1876.0   1876.0             1873.252461          1875.150


In [124]:
# Fehlende Annotationen?
pd.set_option('display.width', 1000)

# Texte, die nicht annotiert werden müssen, z. B. wegen anderer Sprache
exceptions = ['1917.Götze/Ulbricht.085']

texts_to_annotate = (
    meta
    .query("1850 <= year <= 1918")
    .query("corpus == 'anth' or author in @modcanon_authors or author in @muench_authors")
    .query("annotated == False and geschichtslyrik.isna()")
    .query("id not in @exceptions")
    .drop_duplicates(subset='author_title')
    .sort_values(by = 'year')
)

if texts_to_annotate.shape[0] == 0:
    print('Keine fehlenden Annotationen')
else:
    print('Fehlende Annotationen:\n')
    print(texts_to_annotate[['id', 'author', 'title', 'year', 'year_gt', 'year_predict_rfr', 'corpus']])

# texts_to_annotate[['id', 'author', 'title', 'year', 'year_gt', 'corpus']].to_csv("fehlende_annotationen.csv")

Keine fehlenden Annotationen


In [125]:
# Fehlende Digitalisate?
texts_to_digitize_anth = (
    meta
    .query("1870 <= anthology_year_first_ed <= 1945")
    .query("text_bestocr.isna()")
)

texts_to_digitize_add = (
    meta
    .query("1850 <= year <= 1918")
    .query("author in @modcanon_authors or author in @muench_authors")
    .query("text_bestocr.isna()")
)

texts_to_digitize = pd.concat([texts_to_digitize_anth, texts_to_digitize_add])

texts_to_digitize = texts_to_digitize[[
    'author_title', 'year', 'year_gt', 'corpus'
]].drop_duplicates(subset='author_title').sort_values(by = 'author_title')

if texts_to_digitize.shape[0] == 0:
    print('Keine fehlenden Digitalisate')
else:
    print('Fehlende Digitalisate:\n')
    print(texts_to_digitize)

Keine fehlenden Digitalisate


In [126]:
# Anzahl Texte in Geschichtslyrik_anth
results = (meta
    .query("corpus=='anth'")
    .query("1850 <= year <= 1918")
    .query("geschichtslyrik == 1")
    .drop_duplicates(subset='author_title')
)

print(f"total           : {results.shape[0]}")
print(f"manuell datiert : {results['year_gt'].notna().sum()}\n")

print(results.groupby('decade').count()['id'])
print("\nmin_year:")
print(results.groupby('year')['year'].count().reindex(range(1850, 1919), fill_value=0).nsmallest(3))

total           : 1850
manuell datiert : 1432

decade
1850.0    425
1860.0    376
1870.0    396
1880.0    260
1890.0    153
1900.0    140
1910.0    100
Name: id, dtype: int64

min_year:
year
1904    3
1916    6
1918    6
Name: year, dtype: int64


# Export

In [127]:
meta.to_csv(r"../resources/meta.csv", index = False, header = True)
meta.to_json(r"../resources/meta.json")